In [ ]:
import requests, os
from bs4 import BeautifulSoup
import re

In [ ]:
%pwd

In [ ]:
check_url = 'https://digital.nhs.uk/data-and-information/publications/statistical/nhs-workforce-statistics'
file_source_url = 'https://digital.nhs.uk'

response = requests.get(check_url)

soup = BeautifulSoup(response.content, 'html.parser')
links = soup.find( id="past-publications").find_all(href=re.compile("publications/statistical/nhs-workforce-statistics"))

In [ ]:
from datetime import datetime

In [ ]:
def retrieve_stats_urls(url_string, base_url_string = "publications/statistical/nhs-workforce-statistics"):
    x = url_string.split('/')
    #print(x[-1])
    date_string = x[-1]
    try:
        formatted_date = datetime.strptime(date_string, "%B-%Y")
        #print(formatted_date)
        return {
            'orig_url': url_string,
            'the_date': formatted_date
        }
    except:
        #print('Could not format date')
        pass

In [ ]:
stats_urls_list = []

for link in links:
    #print(link['href'])
    get_url = retrieve_stats_urls(link['href'])
    if get_url is not None:
        stats_urls_list.append(get_url)

In [ ]:
stats_urls_list

In [ ]:
stats_urls_list.sort(key = lambda x:x['the_date'], reverse=True)

In [ ]:
web_page_for_data = file_source_url + stats_urls_list[1]['orig_url']

print(web_page_for_data)

#fetch_zip_urls <- read_html(web_page_for_data) %>% html_elements('a') %>% html_attr('href')

In [ ]:
response2 = requests.get("https://digital.nhs.uk/data-and-information/publications/statistical/nhs-workforce-statistics/december-2022")

soup2 = BeautifulSoup(response2.content, 'html.parser')
links2 = soup2.find('a', href=re.compile("Turnover"))['href']
links2

In [ ]:
def fetch_zip_url(url):
    # Iterate through URLs as latest may not have any zip files
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    return(soup.find(href=re.compile(".zip")))

In [ ]:
a = fetch_zip_url("https://digital.nhs.uk/data-and-information/publications/statistical/nhs-workforce-statistics/december-2022")

In [ ]:
a['href']

In [ ]:
from pathlib import Path

for filename in Path("./tempdir").glob("*.*"):
    filename.unlink()

In [ ]:
resp = requests.get(a['href']) # making requests to server

# Check whether the specified path exists or not
isExist = os.path.exists("./tempdir/")
if not isExist:
   # Create a new directory because it does not exist
   os.makedirs("./tempdir/")
   
with open("./tempdir/turnover.zip", "wb") as f: # opening a file handler to create new file 
    f.write(resp.content) # writing content to file
    f.close()

In [ ]:
import zipfile
with zipfile.ZipFile("./tempdir/turnover.zip", 'r') as zip_ref:
    zip_ref.extractall("./tempdir")

In [ ]:
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://directory.spineservices.nhs.uk/ORD/2-0-0/organisations?Status=Inactive&Roles=RO197,RO98&Limit=1000"
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())
  
# print the json response
#print(data_json)

In [ ]:
inactive_orgs = []

for q in data_json['Organisations']:
    inactive_orgs.append({
        'name': q['Name'],
        'org_id' : q['OrgId']
    })

In [ ]:
len(inactive_orgs)

In [ ]:
with open("./tempdir/inactive_organisations.txt", 'w') as f:
    for i in inactive_orgs:
        f.write(f"{i}\n")